# I. introduction

***inferences***: https://viblo.asia/p/shufflenet-deep-network-danh-cho-thiet-bi-mobile-ORNZq1P8Z0n

***các khái niệm cơ bản***

- pointwise convolution: tích chập nhiều điểm

- grouped convolution: tích chập nhóm

- channel shuffle: xáo trộn kênh

- deepwise separable convolution: tích chập có thể phân tách theo chiều sâu

***pointwise convolution: tích chập nhiều điểm***

- là 1x1 convolution.

trong một mô hình học sâu, khi nó tăng lên độ sâu:
    - features map tăng lên theo chiều sâu của mô hình
    - tăng độ phức tạp và tính toán
    - số lượng tham số tăng lên => overfitting

các phương pháp giảm tham số:
    - giảm kích thước của các features map là các lớp pooling: maxpooling, avepooling...    
    => giảm kích thước chiều rộng và cao của một feature map nhưng vẫn giữ được các đặc trưng cần có

* tuy nhiên các lớp pooling không giúp chúng ta tăng/giảm kích thước channel
=> pointwise ra đời để giải quyết vấn đề này


***pointwise convolution***

- có kích thước hai chiều đầu là 1x1, chiều số 3 có kích thước bằng với channel ở input
- ma trận có kích thước: 28*28*192, ta suy ra pointwise convolution có kích thước là 1*1*192.
ta nhân hai số đó với số lượng filter đầu ra mong muốn (ví dụ 32), ta sẽ có đầu ra như sau: 28*28*32
=> chiều dài và chiều rộng đầu vào không thay đổi, chỉ thay đổi channel theo kích thước ý muốn

***grouped convolution***

# II. training

In [1]:
import os
os.environ['JAX_PLATFORMS'] = ''

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from os import listdir
from cv2 import imread
from cv2 import imshow
from cv2 import resize
from keras.applications import EfficientNetB0
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Flatten, Dropout
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint


In [3]:
base_path = "E:/UDA_LEARNING/UDA_DO AN TRI TUE NHAN TAO & UNG DUNG/BREAST CANCER/data/IDC_regular_ps50_idx5"
folder = listdir(base_path)
len(folder)
print(folder[:5])

['10253', '10254', '10255', '10256', '10257']


In [4]:
# how many image input?
total_images = 0
for n in range(len(folder)):
    patient_id = folder[n]
    for c in [0, 1]:
        patient_path = base_path + "/" + patient_id
        class_path = patient_path + "/" + str(c) + "/"
        subfiles = listdir(class_path)
        total_images += len(subfiles)

print(total_images)

277524


In [5]:
# split patient into frame: patient_id, path, target (id patient, path/to/patient and label)
data = pd.DataFrame(index=np.arange(0, total_images), columns=["patient_id", "path", "target"])

k = 0
for n in range(len(folder)):
    patient_id = folder[n]
    patient_path = base_path + "/" + patient_id 
    for c in [0,1]:
        class_path = patient_path + "/" + str(c) + "/"
        subfiles = listdir(class_path)
        for m in range(len(subfiles)):
            image_path = subfiles[m]
            data.loc[k, "path"] = class_path + image_path
            data.loc[k, "target"] = c
            data.loc[k, "patient_id"] = patient_id
            k += 1  

data.head()

,patient_id,path,target
0,10253,E:/UDA_LEARNING/UDA_DO AN TRI TUE NHAN TAO & U...,0
1,10253,E:/UDA_LEARNING/UDA_DO AN TRI TUE NHAN TAO & U...,0
2,10253,E:/UDA_LEARNING/UDA_DO AN TRI TUE NHAN TAO & U...,0
3,10253,E:/UDA_LEARNING/UDA_DO AN TRI TUE NHAN TAO & U...,0
4,10253,E:/UDA_LEARNING/UDA_DO AN TRI TUE NHAN TAO & U...,0


In [6]:
Pos_path = data.groupby("target").get_group(0)
Neg_path = data.groupby('target').get_group(1)

In [7]:
print(Neg_path[0:5])
print(len(Neg_path))

    patient_id                                               path target
479      10253  E:/UDA_LEARNING/UDA_DO AN TRI TUE NHAN TAO & U...      1
480      10253  E:/UDA_LEARNING/UDA_DO AN TRI TUE NHAN TAO & U...      1
481      10253  E:/UDA_LEARNING/UDA_DO AN TRI TUE NHAN TAO & U...      1
482      10253  E:/UDA_LEARNING/UDA_DO AN TRI TUE NHAN TAO & U...      1
483      10253  E:/UDA_LEARNING/UDA_DO AN TRI TUE NHAN TAO & U...      1
78786


In [8]:
lowerIndex = 0
upperIndex = 50000

Pos_train_path = list(Pos_path[lowerIndex: upperIndex]['path'])
Pos_train_target = list(Pos_path[lowerIndex: upperIndex]['target'])

print(len(Pos_train_path))
print(len(Pos_train_target))


50000
50000


In [9]:
Neg_train_path = list(Neg_path[lowerIndex: upperIndex]['path'])
Neg_train_target = list(Neg_path[lowerIndex: upperIndex]['target'])

print(len(Pos_train_path))
print(len(Neg_train_target))

50000
50000


In [10]:
x_train = []
y_train = []
x_test = []
y_test = []

WIDTH = 50
HEIGHT = 50

for path in Pos_train_path:
    origImageSize = imread(path)
    resizedImage = resize(origImageSize, (WIDTH, HEIGHT), interpolation=cv2.INTER_CUBIC)
    x_train.append(resizedImage)

for path in Neg_train_path:
    origImageSize = imread(path)
    resizedImage = resize(origImageSize, (WIDTH, HEIGHT), interpolation=cv2.INTER_CUBIC)
    x_train.append(resizedImage)

In [11]:
print(len(x_train))
y_train = Pos_train_target + Neg_train_target
print(len(y_train))

100000
100000


In [12]:
combined = list(zip(x_train, y_train))
np.random.shuffle(combined)
x_train, y_train = zip(*combined)

In [13]:
x_train = np.array(x_train)/255.0
y_train = np.array(y_train)

In [21]:
import tensorflow as tf
from keras import layers, models

def channel_shuffle(x, groups):
    height, width, channels = x.shape.as_list()[1:]
    channels_per_group = channels // groups
    
    x = tf.reshape(x, [-1, height, width, groups, channels_per_group])
    x = tf.transpose(x, [0, 1, 2, 4, 3])
    x = tf.reshape(x, [-1, height, width, channels])
    
    return x

def shufflenet_unit(inputs, in_channels, out_channels, groups):
    shortcut = inputs
    
    # 1x1 convolution
    x = layers.Conv2D(in_channels, kernel_size=1, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    # Depthwise Convolution
    x = layers.DepthwiseConv2D(kernel_size=3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    
    # Pointwise Convolution
    x = layers.Conv2D(out_channels, kernel_size=1, padding='same')(x)
    x = layers.BatchNormalization()(x)
    
    # Add the shortcut to the output
    if in_channels == out_channels:
        x += shortcut
    else:
        # If the number of channels changes, apply pointwise convolution to the shortcut
        shortcut = layers.Conv2D(out_channels, kernel_size=1, padding='same')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)
        x += shortcut
    
    # Apply channel shuffle
    x = layers.ReLU()(x)
    x = layers.Lambda(channel_shuffle, arguments={'groups': groups}, output_shape=lambda x: x)(x)

    return x

def ShuffleNet(input_shape=(50, 50, 3), groups=3):
    inputs = layers.Input(shape=input_shape)
    
    # Initial Convolution
    x = layers.Conv2D(24, kernel_size=3, strides=2, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    # Max Pooling
    x = layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(x)
    
    # Stages
    stage_repeats = [3, 7, 3]
    stage_out_channels = [144, 288, 576]
    in_channels = 24
    for i, (num_blocks, out_channels) in enumerate(zip(stage_repeats, stage_out_channels)):
        # First block in each stage needs to take care of stride and channels
        x = shufflenet_unit(x, in_channels=in_channels, out_channels=out_channels, groups=groups)
        in_channels = out_channels
        for j in range(num_blocks - 1):
            x = shufflenet_unit(x, in_channels=in_channels, out_channels=out_channels, groups=groups)
    
    # Global Average Pooling
    x = layers.GlobalAveragePooling2D()(x)
    
    # Fully Connected Layer
    x = layers.Dense(1, activation='sigmoid')(x)
    
    model = models.Model(inputs, x)
    return model

# Example of creating ShuffleNet with 3 groups and binary classification
model = ShuffleNet(groups=3)
model.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6       │ (None, 50, 50, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_50 (Conv2D)  │ (None, 25, 25,    │        672 │ input_layer_6[0]… │
│                     │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 25, 25,    │         96 │ conv2d_50[0][0]   │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_42 (ReLU)     │ (None, 25, 25,    │          0 │ batch_normalizat… │
│                     │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_6     │ (None, 13, 13,    │          0 │ re_lu_42[0][0]    │
│ (MaxPooling2D)      │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_51 (Conv2D)  │ (None, 13, 13,    │        600 │ max_pooling2d_6[… │
│                     │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 13, 13,    │         96 │ conv2d_51[0][0]   │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_43 (ReLU)     │ (None, 13, 13,    │          0 │ batch_normalizat… │
│                     │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ depthwise_conv2d_18 │ (None, 13, 13,    │        240 │ re_lu_43[0][0]    │
│ (DepthwiseConv2D)   │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 13, 13,    │         96 │ depthwise_conv2d… │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_52 (Conv2D)  │ (None, 13, 13,    │      3,600 │ batch_normalizat… │
│                     │ 144)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_53 (Conv2D)  │ (None, 13, 13,    │      3,600 │ max_pooling2d_6[… │
│                     │ 144)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 13, 13,    │        576 │ conv2d_52[0][0]   │
│ (BatchNormalizatio… │ 144)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 13, 13,    │        576 │ conv2d_53[0][0]   │
│ (BatchNormalizatio… │ 144)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_18 (Add)        │ (None, 13, 13,    │          0 │ batch_normalizat… │
│                     │ 144)              │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_44 (ReLU)     │ (None, 13, 13,    │          0 │ add_18[0][0]      │
│                     │ 144)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_18 (Lambda)  │ (None, 13, 13,    │          0 │ re_lu_44[0][0]  

 Total params: 3,027,385 (11.55 MB)

 Trainable params: 3,002,473 (11.45 MB)

 Non-trainable params: 24,912 (97.31 KB)

In [22]:
from keras.callbacks import ModelCheckpoint

callBack = ModelCheckpoint(
    filepath = "BreastCancerModel_ShuffleNet.keras",
    save_weights_only=False,
    monitor='accuracy',
    mode='max',
    save_best_only=True,
)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=10, validation_split=0.2)

RuntimeError: Unable to automatically build the model. Please build it yourself before calling fit/evaluate/predict. A model is 'built' when its variables have been created and its `self.built` attribute is True. Usually, calling the model on a batch of data is the right way to build it.
Exception encountered:
'Exception encountered when calling Lambda.call().

[1m'tuple' object has no attribute 'as_list'[0m

Arguments received by Lambda.call():
  • inputs=jnp.ndarray(shape=(32, 13, 13, 144), dtype=float32)
  • mask=None
  • training=None'